In [1]:
import nltk
import random
import pandas as pd
import re

#importieren der Trainigsdaten---nltk stellt tweets dafür bereit
from nltk.corpus import twitter_samples

In [2]:
pos_twee = twitter_samples.strings('positive_tweets.json')
neg_twee = twitter_samples.strings('negative_tweets.json')
text = twitter_samples.strings('tweets.20150430-223406.json')


In [3]:
print(pos_twee[0])

#FollowFriday @France_Inte @PKuchly57 @Milipol_Paris for being top engaged members in my community this week :)


In [4]:
print(len(text))
print(len(pos_twee))
print(len(neg_twee))

20000
5000
5000


In [5]:
df = pd.DataFrame (pos_twee,columns=['text'])
df['Value'] = 1
df2 = pd.DataFrame (neg_twee,columns=['text'])
df2['Value'] = -1

In [6]:
df

,text,Value
0,#FollowFriday @France_Inte @PKuchly57 @Milipol...,1
1,@Lamb2ja Hey James! How odd :/ Please call our...,1
2,@DespiteOfficial we had a listen last night :)...,1
3,@97sides CONGRATS :),1
4,yeaaaah yippppy!!! my accnt verified rqst has...,1
...,...,...
4995,"@chriswiggin3 Chris, that's great to hear :) D...",1
4996,@RachelLiskeard Thanks for the shout-out :) It...,1
4997,@side556 Hey! :) Long time no talk...,1
4998,@staybubbly69 as Matt would say. WELCOME TO AD...,1


In [7]:
df2

,text,Value
0,hopeless for tmr :(,-1
1,Everything in the kids section of IKEA is so c...,-1
2,@Hegelbon That heart sliding into the waste ba...,-1
3,"“@ketchBurning: I hate Japanese call him ""bani...",-1
4,"Dang starting next week I have ""work"" :(",-1
...,...,...
4995,I wanna change my avi but uSanele :(,-1
4996,MY PUPPY BROKE HER FOOT :(,-1
4997,where's all the jaebum baby pictures :((,-1
4998,But but Mr Ahmad Maslan cooks too :( https://t...,-1


In [8]:
df=df.append(df2, ignore_index = True)

In [9]:
df = df.sample(frac=1).reset_index(drop=True) #msichen der positiven und negativen tweets


In [10]:
df


,text,Value
0,@robertsammons it's released online but not on...,-1
1,@Barrastinian @chunkymark What if they had a c...,1
2,@marthasydenham @alexxdouglas @wordpressdotcom...,1
3,@LottyStorer @charliee_catlin 😂😩😩 what a mess :(,-1
4,@_CaitlinTierney dnt stab meh :(,-1
...,...,...
9995,@Hre_Krsna thank you. ..:-),1
9996,@kohquette you're good and you should feel goo...,-1
9997,@Miss_Wisconsin That was cold. :),1
9998,@tookiebunten looks like a few of our customer...,1


# bereinigen der Daten

In [11]:
#bereinigen der Tweets
remove_rt = lambda x: re.sub("(RT @\w+: )|(@\w+):"," ",x) #herausfiltern der usernamen und RT
rt = lambda x: re.sub("(@[A-Za-z0–9]+)|[\d\.]|(\w+:\/\/\S+)|(\n|\t|\')"," ",x) #herausfiltern markierter Usernamen 
htp = lambda x: re.sub('http[s]?://(?:[a-zA-Z]|[0-9]|[$-_@.&+#]|[!*\(\),]|(?:%[0-9a-fA-F][0-9a-fA-F]))+'," ",x) #entfernen von links, sonderzeichen, punkten

df["text"] = df.text.map(remove_rt).map(rt).map(htp)
df["text"] = df.text.str.lower()
df.text.head(10)

0      it s released online but not on the album th...
1                          what if they had a child :d
2          lovely stuff  this needs to be repeated,...
3                           _catlin 😂😩😩 what a mess :(
4                     @_caitlintierney dnt stab meh :(
5        ns                  it s #tellanoldjokeday...
6                                           more :(   
7    how far would you go just to win? camara knows...
8                            sepanx with this one :(  
9     _lesly thank you for the screenshots po  let ...
Name: text, dtype: object

In [12]:
#bereinigen des textes von den Emojis
def deEmojify(text):
    regrex_pattern = re.compile("["
                           u"\U0001F600-\U0001F64F"  # emoticons
                           u"\U0001F300-\U0001F5FF"  # symbols & pictographs
                           u"\U0001F680-\U0001F6FF"  # transport & map symbols
                           u"\U0001F1E0-\U0001F1FF"  # flags (iOS)
                           u"\U00002702-\U000027B0"
                           u"\U000024C2-\U0001F251"
                           "]+", flags=re.UNICODE)
    return regrex_pattern.sub(r'',text)

df["text"] = df.text.map(deEmojify)

df.text.head(5)

0      it s released online but not on the album th...
1                          what if they had a child :d
2          lovely stuff  this needs to be repeated,...
3                              _catlin  what a mess :(
4                     @_caitlintierney dnt stab meh :(
Name: text, dtype: object

In [13]:
data = list(df["text"])
labels = list(df["Value"])

In [14]:
data[:5]

['  it s released online but not on the album that i know of :(',
 '    what if they had a child :d',
 '      lovely stuff  this needs to be repeated, relentlessly, for approximately the next   years  :)',
 '   _catlin  what a mess :(',
 '@_caitlintierney dnt stab meh :(']

In [15]:
from sklearn.pipeline import Pipeline
from sklearn.naive_bayes import MultinomialNB
from sklearn.feature_extraction.text import CountVectorizer, TfidfTransformer,TfidfVectorizer
from sklearn.model_selection import train_test_split, GridSearchCV

In [16]:
x_train, x_test, y_train, y_test = train_test_split(data, labels, test_size=0.33, random_state=42)

In [17]:
x_train[:10]

['this girl :)   ',
 '  yeah, they are just the next prey :( poor girls',
 'join us for see, make do! every wed, thurs &amp; fri in july &amp; aug (excl    aug) free crafts &amp; we ll do the tidying up :)  ',
 'i blogged, here it is :) any excuse will not do!  problem attenders   via  _mvhr',
 'don t even think about it :)))',
 '  happy birthday, miss chenoweth! hope its a great one    have oodles of cake :)',
 'well it seems like just a dream :(',
 'gooood morninggggg and jummah mubarak everyone! have a blessed and amazing day ahead :)',
 'not    hours after my sister-in-law installed that new   seat we bought her she was rear-ended! everyone s fine but seat is trash :(',
 'hi    : jarraaaaa do you want to get free iphone  ? hurry check my bi   thx :)  ']

In [18]:
vectorizer = TfidfVectorizer()

In [19]:
vectors = vectorizer.fit_transform(data)


vectors_train = vectorizer.transform(x_train)

In [20]:
model = MultinomialNB(alpha=.01)
model = model.fit(vectors_train, y_train)

In [21]:
vectors_test = vectorizer.transform(x_test)

# Auswertung des Models

In [22]:
from sklearn.metrics import classification_report

print(classification_report(y_test, model.predict(vectors_test), digits=4))

              precision    recall  f1-score   support

          -1     0.7191    0.7503    0.7344      1682
           1     0.7282    0.6953    0.7113      1618

    accuracy                         0.7233      3300
   macro avg     0.7236    0.7228    0.7229      3300
weighted avg     0.7235    0.7233    0.7231      3300



# Anreichern der Tweets

In [23]:
#einlesen der Daten
data_tweets = pd.read_json("./cryptodoge.json", lines=True)
df_tweets=data_tweets


In [24]:
#bereinigen der Tweets
remove_rt = lambda x: re.sub("(RT @\w+: )|(@\w+):"," ",x) #herausfiltern der usernamen und RT
rt = lambda x: re.sub("(@[A-Za-z0–9]+)|[\d\.]|(\w+:\/\/\S+)|(\n|\t|\')"," ",x) #herausfiltern markierter Usernamen 
htp = lambda x: re.sub('http[s]?://(?:[a-zA-Z]|[0-9]|[$-_@.&+#]|[!*\(\),]|(?:%[0-9a-fA-F][0-9a-fA-F]))+'," ",x) #entfernen von links, sonderzeichen, punkten

df_tweets["text"] = df.text.map(remove_rt).map(rt).map(htp)
df_tweets["text"] = df.text.str.lower()
df_tweets.text.head(10)

#bereinigen des textes von den Emojis
def deEmojify(text):
    regrex_pattern = re.compile("["
                           u"\U0001F600-\U0001F64F"  # emoticons
                           u"\U0001F300-\U0001F5FF"  # symbols & pictographs
                           u"\U0001F680-\U0001F6FF"  # transport & map symbols
                           u"\U0001F1E0-\U0001F1FF"  # flags (iOS)
                           u"\U00002702-\U000027B0"
                           u"\U000024C2-\U0001F251"
                           "]+", flags=re.UNICODE)
    return regrex_pattern.sub(r'',text)

df_tweets["text"] = df_tweets.text.map(deEmojify)

In [25]:
df_tweets[:4]

,created_at,id,id_str,text,source,truncated,in_reply_to_status_id,in_reply_to_status_id_str,in_reply_to_user_id,in_reply_to_user_id_str,...,lang,timestamp_ms,display_text_range,extended_tweet,possibly_sensitive,quoted_status_id,quoted_status_id_str,quoted_status,quoted_status_permalink,extended_entities
0,2021-06-28 16:25:33+00:00,1409548535272407047,1409548535272407040,it s released online but not on the album th...,"<a href=""http://twitter.com/download/android"" ...",False,NaN,NaN,NaN,NaN,...,en,2021-06-28 16:25:33.611,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,2021-06-28 16:25:33+00:00,1409548536740552712,1409548536740552704,what if they had a child :d,"<a href=""http://twitter.com/download/android"" ...",False,NaN,NaN,NaN,NaN,...,en,2021-06-28 16:25:33.961,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,2021-06-28 16:25:34+00:00,1409548538988609537,1409548538988609536,"lovely stuff this needs to be repeated,...","<a href=""http://twitter.com/download/android"" ...",False,1.409546e+18,1.409546e+18,8.055323e+17,8.055323e+17,...,en,2021-06-28 16:25:34.497,"[16, 51]",NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,2021-06-28 16:25:34+00:00,1409548540234194947,1409548540234194944,_catlin what a mess :(,"<a href=""https://mobile.twitter.com"" rel=""nofo...",True,1.406900e+18,1.406900e+18,1.074695e+18,1.074695e+18,...,en,2021-06-28 16:25:34.794,"[13, 140]",{'full_text': '@BitcoinAssn The most powerful ...,0.0,NaN,NaN,NaN,NaN,NaN


In [26]:
clean_data_list = list(df_tweets["text"])

In [27]:
clean_data_list[:5]

['  it s released online but not on the album that i know of :(',
 '    what if they had a child :d',
 '      lovely stuff  this needs to be repeated, relentlessly, for approximately the next   years  :)',
 '   _catlin  what a mess :(',
 '@_caitlintierney dnt stab meh :(']

In [28]:
#kalkulieren der Wahrscheinlichkeiten des jeweiligen Textes

In [29]:
one_test = vectorizer.transform(list(clean_data_list))

In [50]:
prob_single=model.predict_proba(one_test)
prob_single

array([[0.74931223, 0.25068777],
       [0.48492247, 0.51507753],
       [0.00476579, 0.99523421],
       ...,
       [0.29384447, 0.70615553],
       [0.04261569, 0.95738431],
       [0.03773396, 0.96226604]])

In [58]:
neg_prob=[]
pos_prob=[]
print(type(neg_prob))

for i in prob_single:
    
    neg_prob.append(i[0])
    pos_prob.append(i[1])



<class 'list'>


In [62]:
data_tweets["neg_prob"]=neg_prob
data_tweets["pos_prob"]=pos_prob

In [67]:
data_tweets[:5]

,created_at,id,id_str,text,source,truncated,in_reply_to_status_id,in_reply_to_status_id_str,in_reply_to_user_id,in_reply_to_user_id_str,...,extended_tweet,possibly_sensitive,quoted_status_id,quoted_status_id_str,quoted_status,quoted_status_permalink,extended_entities,neg_prob,pos_prob,sentiment
0,2021-06-28 16:25:33+00:00,1409548535272407047,1409548535272407040,it s released online but not on the album th...,"<a href=""http://twitter.com/download/android"" ...",False,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.749312,0.250688,negative
1,2021-06-28 16:25:33+00:00,1409548536740552712,1409548536740552704,what if they had a child :d,"<a href=""http://twitter.com/download/android"" ...",False,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.484922,0.515078,neutral
2,2021-06-28 16:25:34+00:00,1409548538988609537,1409548538988609536,"lovely stuff this needs to be repeated,...","<a href=""http://twitter.com/download/android"" ...",False,1.409546e+18,1.409546e+18,8.055323e+17,8.055323e+17,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.004766,0.995234,positive
3,2021-06-28 16:25:34+00:00,1409548540234194947,1409548540234194944,_catlin what a mess :(,"<a href=""https://mobile.twitter.com"" rel=""nofo...",True,1.406900e+18,1.406900e+18,1.074695e+18,1.074695e+18,...,{'full_text': '@BitcoinAssn The most powerful ...,0.0,NaN,NaN,NaN,NaN,NaN,0.955738,0.044262,negative
4,2021-06-28 16:25:34+00:00,1409548540494245896,1409548540494245888,@_caitlintierney dnt stab meh :(,"<a href=""http://twitter.com/download/android"" ...",False,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.998355,0.001645,negative


In [66]:
data_tweets["sentiment"]= 888

for index, neg in data_tweets["neg_prob"].iteritems():
    
    
    if neg > 0.55 :
        data_tweets.loc[index,"sentiment"]="negative"

    elif neg < 0.45:
        data_tweets.loc[index,"sentiment"]= "positive"

    else:
        data_tweets.loc[index,"sentiment"]= "neutral"
        
    
 

In [68]:
data_tweets[:5]

,created_at,id,id_str,text,source,truncated,in_reply_to_status_id,in_reply_to_status_id_str,in_reply_to_user_id,in_reply_to_user_id_str,...,extended_tweet,possibly_sensitive,quoted_status_id,quoted_status_id_str,quoted_status,quoted_status_permalink,extended_entities,neg_prob,pos_prob,sentiment
0,2021-06-28 16:25:33+00:00,1409548535272407047,1409548535272407040,it s released online but not on the album th...,"<a href=""http://twitter.com/download/android"" ...",False,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.749312,0.250688,negative
1,2021-06-28 16:25:33+00:00,1409548536740552712,1409548536740552704,what if they had a child :d,"<a href=""http://twitter.com/download/android"" ...",False,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.484922,0.515078,neutral
2,2021-06-28 16:25:34+00:00,1409548538988609537,1409548538988609536,"lovely stuff this needs to be repeated,...","<a href=""http://twitter.com/download/android"" ...",False,1.409546e+18,1.409546e+18,8.055323e+17,8.055323e+17,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.004766,0.995234,positive
3,2021-06-28 16:25:34+00:00,1409548540234194947,1409548540234194944,_catlin what a mess :(,"<a href=""https://mobile.twitter.com"" rel=""nofo...",True,1.406900e+18,1.406900e+18,1.074695e+18,1.074695e+18,...,{'full_text': '@BitcoinAssn The most powerful ...,0.0,NaN,NaN,NaN,NaN,NaN,0.955738,0.044262,negative
4,2021-06-28 16:25:34+00:00,1409548540494245896,1409548540494245888,@_caitlintierney dnt stab meh :(,"<a href=""http://twitter.com/download/android"" ...",False,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.998355,0.001645,negative
